# **과제1 : 챗봇만들기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기1
    * Vector DB 준비하기
    * Retriever, LLM를 연결하기

## **1.환경준비**

### (1) 라이브러리 Import

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [5]:
# os.environ['OPENAI_API_KEY'] = '내 API Key'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩

In [3]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
# 한문, 한글, 일본어 등 한 글자 표현하는 데 2byte 드는 문자들: UTF-8로 읽어와야 함
data.head()
# 구분은 메타데이터에

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


In [4]:
print(len(data))

10


* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : ./db



In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",  # 경로 지정(.은 default(root) 경로)
                  embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* 데이터 입력
    * DF to Vector DB

In [6]:
data

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...
5,모집/선발,사업자도 지원할 수 있나요?\n사업자는 자영업자로 분류됩니다. 자영업자의 경우 관할...
6,모집/선발,"프리랜서도 지원할 수 있나요?\n고용보험에 가입하지 않은 경우 미취업자로 보지만, ..."
7,모집/선발,"현재 군 복무중인 상태인데, 지원이 가능한가요?\n현재 군 복무중인 상태여도 지원 ..."
8,모집/선발,"퇴사 후 실업급여 수령 중인데, 지원할 수 있나요?\n퇴사 후 고용보험 상실이 완료..."
9,모집/선발,다른 K-DT 과정을 들었는데 지원할 수 있나요?\nK-DT 교육과정은 최초 1회에...


In [7]:
text_list = data['QA'].tolist()
meta_list = data['구분'].tolist()

# meta_list의 각 항목을 딕셔너리로 변환
doc = [Document(page_content=text_list[i], metadata={'구분': meta_list[i]}) for i in range(len(data))]

# 데이터베이스에 문서를 추가
ind = database.add_documents(doc)

* 입력된 데이터 조회

In [8]:
database.get(ind)

{'ids': ['5a4d0122-ded3-4289-8696-83de8e595198',
  '717d7e62-414c-4478-9aae-cd3e613667c2',
  '996adb6f-5dfb-41e7-9946-a75a176d277c',
  '9dcc7f1c-f2eb-476f-8f68-339f55cdcb8a',
  'd09623c6-1871-4c1b-b07e-c973c9cec113',
  'e4515472-56ab-4896-9f73-6476c3e785d5',
  'e524b941-fa6f-4ccc-a359-514d21f7a268',
  'ee824fa1-67c9-42a6-a801-560138d80fde',
  'f2c8e9d3-c5ea-4bee-ba7a-42d17be8ddd6',
  'f6174901-e4f3-4bf7-84e8-03ef6aaec38d'],
 'embeddings': None,
 'metadatas': [{'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'},
  {'구분': '모집/선발'}],
 'documents': ['미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.',
  '35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)',
  '아르바이트를 하고 있는데 지원할

## **3.RAG+LLM모델**

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [9]:
chat = ChatOpenAI(model="gpt-3.5-turbo",
                  temperature = 1.0,
                  max_tokens = 200)

* 모델 선언

In [10]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

# 체인으로 묶음
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True,)
# return_source_documents = False로 하면 크레딧 절약할 수 있음

* 모델 확인

In [11]:
query =  "에이블스쿨이 뭐야?"
result = qa(query)

print(result["result"])

에이블스쿨은 KT에서 미취업자를 대상으로 하는 교육 프로그램입니다. 근로중인 사람은 해당 교육을 받을 수 없으며, 졸업자 및 졸업예정자를 대상으로 합니다. 또한, AI개발자 Track의 경우에는 기본적인 코딩 역량이 필요합니다.


In [12]:
# result는 딕셔너리 형태
result.keys() # query, result, source_documents

dict_keys(['query', 'result', 'source_documents'])

In [13]:
similarities = database.similarity_search_with_score(query, k = k) # 데이터베이스에서 유사도가 높은 문서를 가져옴
print(similarities)
print('-'*50)
for doc in similarities:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.', metadata={'구분': '모집/선발'}), 0.36260028063069394), (Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.', metadata={'구분': '모집/선발'}), 0.38287278442151507), (Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.', metadata={'구분': '모집/선발'}), 0.46540285635054235)]
--------------------------------------------------
유사도 점수 : 0.3626, 문서 내용: 직장인도 지원할 수 있나요?
KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.
유사도 점수 : 0.38287, 문서 내용: 최종 학력 또는 전공과 관계없이 지원할 수 있나요?
KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.
유사도 점수 : 0.4654, 문